In [ ]:
import logging, os

logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import tensorflow as tf

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.get_logger().setLevel('ERROR')

import warnings

warnings.filterwarnings('ignore')

In [ ]:
# def read_requirements_file(file_path="requirements.txt"):
#     """
#     Reads a requirements.txt file and prints each line.
    
#     Args:
#         file_path (str): The path to the requirements.txt file.
#     """
#     req = []
#     try:
#         with open(file_path, 'r') as f:
#             for line in f:
#                 # Remove leading/trailing whitespace and print the line
#                 req.append(line.strip())
        
#     except FileNotFoundError:
#         print(f"Error: The file '{file_path}' was not found.")
#     except Exception as e:
#         print(f"An error occurred: {e}")
#     return req

# # Example usage:
# read_requirements_file() 
# # If your requirements.txt is in a different location, specify the path:
# # read_requirements_file("path/to/your/requirements.txt")

In [ ]:
# from google.cloud import aiplatform

# PROJECT_ID = "strange-metrics-266115"
# LOCATION = "us-central1"
# STAGING_BUCKET = "gs://simplicialcomplex-inputdata"  # your bucket

# aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=STAGING_BUCKET)

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./strange-metrics-266115-d7ca05a26868.json"

In [ ]:
# from google.cloud import storage

# def upload_to_gcs(bucket_name, source_file, destination_blob):
#         """Upload a file to a GCS bucket."""
#         client = storage.Client()
#         bucket = client.bucket(bucket_name)
#         blob = bucket.blob(destination_blob)
#         blob.upload_from_filename(source_file)
#         print(f"✅ Uploaded {destination_blob} to gs://{bucket_name}/{destination_blob}")
# GCS_BUCKET = "simplicialcomplex-outputbucket"

# upload_to_gcs(GCS_BUCKET, "myData.npz", "data/myData.npz")

In [ ]:
# with np.load("myData.npz") as data:
#     myData = data['myData']
#     myData2 = data['myData2']
#     myY = data['myY']
#     myActors = data['myActors']
#     print(np.unique(myY))
#     print(np.unique(myActors))

In [ ]:
# from google.cloud import storage

# # Initialize client
# client = storage.Client(project="strange-metrics-266115")

# # Reference the bucket
# bucket_name = "simplicialcomplex-inputdata"
# bucket = client.bucket(bucket_name)

# # List blobs and count
# blobs = client.list_blobs(bucket_name, prefix='savefiles')
# count = sum(1 for _ in blobs)
# print(f"Number of objects in bucket '{bucket_name}': {count}")

In [ ]:
# from google.cloud import aiplatform

# aiplatform.init(project="strange-metrics-266115", location="us-central1")

# job = aiplatform.CustomJob.from_local_script(
#     display_name="my-tf-job",
#     script_path="fitPersistencesCombinedModelTorchCloud.py",
#     container_uri="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-7:latest",
#     staging_bucket="gs://simplicialcomplex-outputbucket",
#     requirements=["seaborn", "torchmetrics"],
#     machine_type="n1-standard-4",
#     accelerator_type="NVIDIA_TESLA_T4", 
#     accelerator_count=1 
# )

# job.run()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly
import re
from scipy.io import wavfile
from scipy.fftpack import fft
import cv2
import os
import librosa
import librosa.display
import IPython.display as ipd
from glob import glob
import skimage
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
# import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping,ModelCheckpoint,LearningRateScheduler
import tensorflow.keras as keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tqdm import tqdm
import tempfile
import plotly.graph_objects as go

In [ ]:
from gudhi import RipsComplex
import gudhi as gd
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics import pairwise_distances
from scipy.stats import wasserstein_distance
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from gudhi.representations import Landscape
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
from gudhi.representations import PersistenceImage 
from pathos.multiprocessing import ProcessingPool as Pool
from multiprocessing import cpu_count
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

from pydub import AudioSegment
from pydub.utils import which
os.environ["PATH"] += os.pathsep + "/opt/homebrew/bin"
AudioSegment.converter = "/opt/homebrew/bin/ffmpeg"
AudioSegment.ffprobe = "/opt/homebrew/bin/ffprobe"

In [ ]:
# import kagglehub

# # Download latest version
# path_radvess = kagglehub.dataset_download("uwrfkaggler/ravdess-emotional-speech-audio")

# print("Path to dataset files:", path_radvess)

In [ ]:
# import kagglehub

# # Download latest version
# path_cremad = kagglehub.dataset_download("ejlok1/cremad")

# print("Path to dataset files:", path_cremad)

In [ ]:
# import kagglehub

# # Download latest version
# path_savee = kagglehub.dataset_download("ejlok1/surrey-audiovisual-expressed-emotion-savee")

# print("Path to dataset files:", path_savee)

In [ ]:
# import kagglehub

# # Download latest version
# path_tess = kagglehub.dataset_download("ejlok1/toronto-emotional-speech-set-tess")

# print("Path to dataset files:", path_tess)

In [ ]:
def add_noise(sound: AudioSegment, noise_level=0.02):
    """
    Add Gaussian noise to a pydub AudioSegment.
    noise_level: fraction of max amplitude (0.0 to 1.0)
    """
    samples = np.array(sound.get_array_of_samples()).astype(np.float32)
    # Normalize to [-1, 1]
    samples /= np.iinfo(sound.array_type).max
    # Add Gaussian noise
    noise = np.random.normal(0, noise_level, size=samples.shape)
    noisy_samples = samples + noise
    # Clip to [-1, 1]
    noisy_samples = np.clip(noisy_samples, -1.0, 1.0)
    # Convert back to original dtype
    noisy_samples = (noisy_samples * np.iinfo(sound.array_type).max).astype(sound.array_type)
    noisy_sound = sound._spawn(noisy_samples.tobytes())
    return noisy_sound

In [ ]:
myPath_radvess = 'radvess/1/'
myEmotionMapRadvess = {
    1: 'neutral', 2: 'calm', 3: 'happy', 4: 'sad', 5: 'angry', 6: 'fearful', 7: 'disgust', 8: 'surprised'
}
def loadFiles_radvess(aDirectory):
    myEmotionsSounds, myActors, myEmotions = [], [], []
    for myFolder in tqdm(os.listdir(aDirectory)):
        for myFile in tqdm(os.listdir(aDirectory + myFolder)):
            if myFile.endswith(".wav"):
                myEmotion = myEmotionMapRadvess[int(myFile.split('-')[2])]
                for i in range(2):
                    if myEmotion != 'calm' and myEmotion != 'surprised':
                        sound = AudioSegment.from_file(os.path.join(aDirectory + myFolder, myFile))
                        if i % 2 == 0:
                            sound = add_noise(sound, noise_level=0.2)
                        myEmotionsSounds.append(sound)
                        myActors.append(myFolder)
                        myEmotions.append(myEmotion)
    return myEmotionsSounds, myActors, myEmotions

In [ ]:
myPath_cremad = 'cremad/1/AudioWAV/'
myEmotionMapCremad = {
    'NEU': 'neutral', 'HAP': 'happy', 'SAD': 'sad', 'ANG': 'angry', 'FEA': 'fearful', 'DIS': 'disgust'
}
def loadFiles_cremad(aDirectory):
    myEmotionsSounds, myActors, myEmotions = [], [], []
    for myFile in tqdm(os.listdir(aDirectory)):
        if myFile.endswith(".wav"):
            myEmotion = myEmotionMapCremad[(myFile.split('_')[2].split('.')[0])]
            if myEmotion != 'calm' and myEmotion != 'surprised':
                for i in range(2):
                    myActor = myFile.split('_')[0]
                    sound = AudioSegment.from_file(os.path.join(aDirectory, myFile))
                    if i % 2 == 0:
                        sound = add_noise(sound, noise_level=0.2)
                    myEmotionsSounds.append(sound)
                    myActors.append(myActor)
                    myEmotions.append(myEmotion)
    return myEmotionsSounds, myActors, myEmotions

In [ ]:
myPath_savee = 'savee/1/ALL/'
myEmotionMapSavee = {
    'n': 'neutral', 'h': 'happy', 'sa': 'sad', 'a': 'angry', 'f': 'fearful', 'd': 'disgust', 'su': 'surprised'
}
def loadFiles_savee(aDirectory):
    myEmotionsSounds, myActors, myEmotions = [], [], []
    for myFile in tqdm(os.listdir(aDirectory)):
        if myFile.endswith(".wav"):
            myEmotion = myEmotionMapSavee[(myFile.split('_')[1].split('.')[0])[:-2]]
            if myEmotion != 'calm' and myEmotion != 'surprised':
                for i in range(2):
                    myActor = myFile.split('_')[0]
                    sound = AudioSegment.from_file(os.path.join(aDirectory, myFile))
                    if i % 2 == 0:
                        sound = add_noise(sound, noise_level=0.2)
                    myEmotionsSounds.append(sound)
                    myActors.append(myActor)
                    myEmotions.append(myEmotion)
    return myEmotionsSounds, myActors, myEmotions

In [ ]:
myPath_tess = 'tess/1/TESS Toronto emotional speech set data/'
myEmotionMapTess = {
    'neutral': 'neutral', 'happy': 'happy', 'sad': 'sad', 'angry': 'angry', 'fear': 'fearful', 'disgust': 'disgust', 'pleasant': 'surprised', 'pleasant_surprised': 'surprised'
}
def loadFiles_tess(aDirectory):
    myEmotionsSounds, myActors, myEmotions = [], [], []
    for myFolder in tqdm(os.listdir(aDirectory)):
        if len(myFolder.split('_')) >= 2:
            myActor = myFolder.split('_')[0]
            myEmotion = myEmotionMapTess[myFolder.split('_')[1].lower()]
            if myEmotion != 'calm' and myEmotion != 'surprised':
                for myFile in tqdm(os.listdir(aDirectory + myFolder)): 
                    if myFile.endswith(".wav"):
                        for i in range(2):
                            sound = AudioSegment.from_file(os.path.join(aDirectory + myFolder, myFile))
                            if i % 2 == 0:
                                sound = add_noise(sound, noise_level=0.2)
                            myEmotionsSounds.append(sound)
                            myActors.append(myActor)
                            myEmotions.append(myEmotion)
    return myEmotionsSounds, myActors, myEmotions

In [ ]:
def getWaveform(aPath, aPathName):
    mySignal, mySampleRate = librosa.load(aPath)
    plt.figure(figsize=(14, 5))
    librosa.display.waveshow(mySignal, sr=mySampleRate)
    plt.title(F'The {aPathName} Sound Waveform')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')
    plt.show()
    print(F"The {aPathName} Sound: ")
    sound = ipd.Audio(mySignal, rate=mySampleRate)
    return sound

In [ ]:
def doMelSpectrogram(aSoundFiles, aFixLength = False):
    mySampleRate = 44100  
    myMelSpectrograms = []
    for i in tqdm(range(len(aSoundFiles))):
        myAudioSegment = aSoundFiles[i]
        myTempFile = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        myTempFilePath = myTempFile.name
        myTempFile.close()
        myAudioSegment.export(myTempFilePath, format="wav")
        mySignal, mySampleRate = librosa.load(myTempFilePath, sr=mySampleRate)
        os.remove(myTempFilePath)
        mel_spectrogram = librosa.feature.melspectrogram(y=mySignal, sr=mySampleRate, n_mels=128) 
        mel_spectrogram_dbs = librosa.amplitude_to_db(mel_spectrogram, ref=np.max)
        if aFixLength:
            mel_spectrogram_dbs = librosa.util.fix_length(mel_spectrogram_dbs, size=256, axis=1)
        myMelSpectrograms.append(mel_spectrogram_dbs)
    return myMelSpectrograms

In [ ]:
def computeMfccs(aDbs, aNumMfcc=13):
    myMfccList = []
    for i in tqdm(range(len(aDbs))):
        myMfcc = librosa.feature.mfcc(S=aDbs[i], n_mfcc=aNumMfcc)
        myMfccList.append(myMfcc)
    return myMfccList

In [ ]:
def showMelSpectrogram(aMelSpectrogram):  
    myDfMelSpectrogram = pd.DataFrame(aMelSpectrogram)
    plt.figure(figsize=(10, 4))
    sns.heatmap(myDfMelSpectrogram, cmap='plasma', xticklabels=10, yticklabels=10)
    plt.title(f'Mel Spectrogram from an angry sound  (dB)')
    plt.xlabel('Time (s)')
    plt.ylabel('Mel Frequency Bin')
    plt.show()

In [ ]:
def showMfcc(aMfcc):
    librosa.display.specshow(aMfcc, x_axis='time')
    plt.colorbar()
    plt.title('MFCC From an Angry Sound')
    plt.show()

In [ ]:
def show3DPlot(aMelSpectrogram):
    fig = go.Figure(data=[
        go.Surface(z=aMelSpectrogram.T, colorscale='viridis')
    ])
    fig.update_layout(
        title='Mel Spectrogram from and angry sound 3-D plot (dB)',
        scene=dict(
            xaxis=dict(title='Time (s)'),
            yaxis=dict(title='Mel Frequency Bin'),
            zaxis=dict(title='Amplitude (dB)')
        )
    )
    fig.show()

## Preprocessing: Getting MFCCs and Spectrograms

In [ ]:
myRadvessSounds, myRadvessActors, myRadvessEmotions = loadFiles_radvess(myPath_radvess)
myCremadSounds, myCremadActors, myCremadEmotions = loadFiles_cremad(myPath_cremad)
mySaveeSounds, mySaveeActors, mySaveeEmotions = loadFiles_savee(myPath_savee)
myTessSounds, myTessActors, myTessEmotions = loadFiles_tess(myPath_tess)


In [ ]:
myMelSpectrogramsRadvess = doMelSpectrogram(myRadvessSounds)
myMelSpectrogramsCremad = doMelSpectrogram(myCremadSounds)
myMelSpectrogramsSavee = doMelSpectrogram(mySaveeSounds)
myMelSpectrogramsTess = doMelSpectrogram(myTessSounds)

In [ ]:
myMfccRadvess = computeMfccs(myMelSpectrogramsRadvess)
myMfccCremad = computeMfccs(myMelSpectrogramsCremad)
myMfccSavee = computeMfccs(myMelSpectrogramsSavee)
myMfccTess = computeMfccs(myMelSpectrogramsTess)

## Computing Homologies!

In [ ]:
def euclideanMetric(x, y):
    return np.linalg.norm(x - y) 
def timeBasedEuclideanMetric(x, y):
    return np.linalg.norm(x[0] - y[0]) / (2) + np.linalg.norm(x[1:] - y[1:]) / (2)


In [ ]:
def computerPersistenceDiagrams(aMatrix, aMaxLength, aMaxDim, aMetric, aPlot, aNormalize):
    # Preprocess matrix
    if aNormalize:
        myTimeRow = np.arange(aMatrix.shape[1]) 
        myNormalizedMatrix = (aMatrix - np.min(aMatrix)) / (np.max(aMatrix) - np.min(aMatrix))
        myTimeRow = myTimeRow / aMatrix.shape[1]
        myNewArr = np.vstack([myTimeRow, myNormalizedMatrix])
    else:
        myNewArr = aMatrix
    
    # Compute distance matrix
    myDistanceMatrix = squareform(pdist(myNewArr.T, metric=aMetric))
    myMaxLength = min(np.abs(myDistanceMatrix).max().max() / 3, aMaxLength)

    # Compute persistences
    myRipsComplex = RipsComplex(points=myNewArr.T, max_edge_length=myMaxLength, distance_matrix=myDistanceMatrix)
    mySimplexTree = myRipsComplex.create_simplex_tree(max_dimension=aMaxDim)
    mySimplexTree.compute_persistence()
    myPersistence = mySimplexTree.persistence()

    # Compute bd matrix
    myBd = []
    for i in range(aMaxDim):
        myBdMatrix = np.array([
            [birth, death] for dim, (birth, death) in myPersistence if dim == i and death != float('inf')
        ])
        myBd.append(myBdMatrix)

    # Plot if needed
    if aPlot:
        gd.plot_persistence_diagram(mySimplexTree.persistence())

    return myBd

In [ ]:
def computerPersistenceDiagramsStar(args):
    return computerPersistenceDiagrams(*args)
def computePersistanceFromDb(aDb, aMaxLength, aMaxDim, aMetric, aPlot, aMaxDiagrams, aNormalize):
    myMat = []
    args = []
    for i, myCurr in enumerate(aDb):
        if i == aMaxDiagrams:
            break
        args.append((myCurr, aMaxLength, aMaxDim, aMetric, aPlot, aNormalize))
    
    with Pool(processes=cpu_count()) as pool:
        myMat = list(
            tqdm(
                pool.imap(computerPersistenceDiagramsStar, args),
                total=len(args),
                desc="Computing persistences"
            )
        )
    return myMat

In [ ]:

def getPersistenceHeatMapSingle(myBdMatrix, aMax, aDim, aPlot, aShape):
    # print(myBdMatrix)
    myPersistenceImage = PersistenceImage(
        bandwidth= aMax / 50,
        weight=lambda x: np.sqrt((x[1] - x[0])**2), 
        resolution=[aShape, aShape],    
        im_range=[0, aMax, 0, aMax]   
    )

    # If any persistence bd matrix found then compute heatmap
    if aDim < len(myBdMatrix) and len(myBdMatrix[aDim]) > 0:
        myHeatMap = myPersistenceImage.fit_transform([myBdMatrix[aDim]])[0]
        myHeatMap2d = myHeatMap.reshape(aShape, aShape)
        if aPlot:
            print(myHeatMap2d.shape)
            plt.figure()
            plt.imshow(myHeatMap2d, cmap='hot', origin='lower')
            plt.colorbar(label='Scaled Density')
            plt.title('Persistence Image')
            plt.xlabel('Birth')
            plt.ylabel('Persistence')
            plt.show()
        return myHeatMap2d
    else:
        return np.zeros((aShape, aShape))
        
def getPersistenceHeatMapSingleStar(args):
    return getPersistenceHeatMapSingle(*args)

def getPersitanceHeatMap(aArr, aMax, aMaxDim, aPlot = False, aShape = 32):
    myHeatmapArr = []
    args = []
    for i, myCurr in enumerate(aArr):
        for j in range(aMaxDim):
            args.append((myCurr, aMax, j, aPlot, aShape))
    
    with Pool(processes=cpu_count()) as pool:
        myHeatmapArr = list(
            tqdm(
                pool.imap(getPersistenceHeatMapSingleStar, args),
                total=len(args),
                desc="Computing persistences heat map"
            )
        )
    return myHeatmapArr

In [ ]:
myPersistenceT = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsCremad[0], aMaxLength=1.2, aMaxDim=2, aMetric=timeBasedEuclideanMetric, aPlot=True, aNormalize=True)
myPersistenceW = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsRadvess[0], aMaxLength=2.5, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)
myPersistenceE = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsSavee[0], aMaxLength=100, aMaxDim=2, aMetric=euclideanMetric, aPlot=True, aNormalize=False)
myPersistenceWM = computerPersistenceDiagrams(aMatrix=myMfccTess[0], aMaxLength=15, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)

heatmaps = getPersitanceHeatMap(aArr=[myPersistenceT], aMax=0.8, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceW], aMax=2, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceE], aMax=80, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceWM], aMax=10, aMaxDim=2, aPlot=True)

In [ ]:
myPersistenceT = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsCremad[0], aMaxLength=0.8, aMaxDim=2, aMetric=timeBasedEuclideanMetric, aPlot=True, aNormalize=True)
myPersistenceW = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsSavee[0], aMaxLength=2, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)
myPersistenceE = computerPersistenceDiagrams(aMatrix=myMelSpectrogramsTess[0], aMaxLength=80, aMaxDim=2, aMetric=euclideanMetric, aPlot=True, aNormalize=False)
myPersistenceWM = computerPersistenceDiagrams(aMatrix=myMfccRadvess[0], aMaxLength=10, aMaxDim=2, aMetric=wasserstein_distance, aPlot=True, aNormalize=False)

heatmaps = getPersitanceHeatMap(aArr=[myPersistenceT], aMax=0.8, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceW], aMax=2, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceE], aMax=80, aMaxDim=2, aPlot=True)
heatmaps = getPersitanceHeatMap(aArr=[myPersistenceWM], aMax=10, aMaxDim=2, aPlot=True)

In [ ]:
def parallelComputePersitence(args):
    return computePersistanceFromDb(*args)

In [ ]:
def saveResults(aPattern, aResults, aMaxDim, dataset, actors, emotions):
    for i in range(len(aResults)):
        for k in range(len(aResults[i])):
            myDataset = dataset[i]
            myActor = actors[i][k]
            myEmotion = emotions[i][k]
            for j in range(aMaxDim):
                np.save(f'./savefiles/{aPattern}_{myDataset}_{myActor}_{myEmotion}_{i}_{k}_{j}.npy', aResults[i][k][j])

In [ ]:
def computeHeatMaps(aResults, aMaxDim, aMaxDist, aName, dataset, actors, emotions):
    myNewHeat = []
    for i in range(len(aResults)):
        # print(f'Computing {i}')
        myHeat = getPersitanceHeatMap(aResults[i], aMax=aMaxDist, aMaxDim=aMaxDim)
        myNewHeat.append(myHeat)
        for j in range(len(myNewHeat[i])):
            # print(j)
            myDataset = dataset[i]
            myActor = actors[i][j // 2]
            myEmotion = emotions[i][j // 2]
            np.save(f'./savefiles/{aName}Heat_{myDataset}_{myActor}_{myEmotion}_{i}_{j}.npy', myNewHeat[i][j])

In [ ]:
maxdim = 2
metrics = [timeBasedEuclideanMetric, wasserstein_distance, euclideanMetric]
scaled = [True, False, False]
maxdists = [0.8, 2, 80]
names = ['timeMetric', 'wasserstein', 'euclidean']
for metric, scale, maxdist, name in zip(metrics, scaled, maxdists, names):
    args = [
        (myMelSpectrogramsCremad, maxdist, maxdim, metric, False, len(myMelSpectrogramsCremad), scale),
        (myMelSpectrogramsSavee, maxdist, maxdim, metric, False, len(myMelSpectrogramsSavee), scale),
        (myMelSpectrogramsRadvess, maxdist, maxdim, metric, False, len(myMelSpectrogramsRadvess), scale),
        (myMelSpectrogramsTess, maxdist, maxdim, metric, False, len(myMelSpectrogramsTess), scale),
    ]
    results = []
    for arg in args:
        results.append(parallelComputePersitence(arg))
    saveResults(aPattern=name, aResults=results, aMaxDim=maxdim, dataset=['cremad', 'savee', 'radvess', 'tess'], actors=[myCremadActors, mySaveeActors, myRadvessActors, myTessActors], emotions=[myCremadEmotions, mySaveeEmotions, myRadvessEmotions, myTessEmotions])
    computeHeatMaps(aResults=results, aMaxDim=maxdim, aMaxDist=maxdist, aName=name, dataset=['cremad', 'savee', 'radvess', 'tess'], actors=[myCremadActors, mySaveeActors, myRadvessActors, myTessActors], emotions=[myCremadEmotions, mySaveeEmotions, myRadvessEmotions, myTessEmotions])
    print('To the next!')

In [ ]:
maxDim = 2
maxdist = 15
args = [
    (myMfccCremad, maxdist, maxdim, metric, False, len(myMfccCremad), scale),
    (myMfccSavee, maxdist, maxdim, metric, False, len(myMfccSavee), scale),
    (myMfccRadvess, maxdist, maxdim, metric, False, len(myMfccRadvess), scale),
    (myMfccTess, maxdist, maxdim, metric, False, len(myMfccTess), scale),
]
results = []
for arg in args:
    results.append(parallelComputePersitence(arg))
saveResults(aPattern='wassersteinMfcc', aResults=results, aMaxDim=maxDim, dataset=['cremad', 'savee', 'radvess', 'tess'], actors=[myCremadActors, mySaveeActors, myRadvessActors, myTessActors], emotions=[myCremadEmotions, mySaveeEmotions, myRadvessEmotions, myTessEmotions])
computeHeatMaps(aResults=results, aMaxDim=maxDim, aMaxDist=maxdist, aName='wassersteinMfcc', dataset=['cremad', 'savee', 'radvess', 'tess'], actors=[myCremadActors, mySaveeActors, myRadvessActors, myTessActors], emotions=[myCremadEmotions, mySaveeEmotions, myRadvessEmotions, myTessEmotions])


In [ ]:
myMelSpectrogramsCremad = doMelSpectrogram(myCremadSounds, True)
myMelSpectrogramsRadvess = doMelSpectrogram(myRadvessSounds, True)
myMelSpectrogramsSavee = doMelSpectrogram(mySaveeSounds, True)
myMelSpectrogramsTess = doMelSpectrogram(myTessSounds, True)

In [ ]:
for i in range(len(myMelSpectrogramsCremad)):
    np.save(f'./savefiles/cremad_{myCremadActors[i]}_{myCremadEmotions[i]}_{i}.npy', myMelSpectrogramsCremad[i])
for i in range(len(myMelSpectrogramsRadvess)):
    np.save(f'./savefiles/radvess_{myRadvessActors[i]}_{myRadvessEmotions[i]}_{i}.npy', myMelSpectrogramsRadvess[i])
for i in range(len(myMelSpectrogramsSavee)):
    np.save(f'./savefiles/savee_{mySaveeActors[i]}_{mySaveeEmotions[i]}_{i}.npy', myMelSpectrogramsSavee[i])
for i in range(len(myMelSpectrogramsTess)):
    np.save(f'./savefiles/tess_{myTessActors[i]}_{myTessEmotions[i]}_{i}.npy', myMelSpectrogramsTess[i])

In [ ]:
folder = './savefiles'

def findFilesFromPattern(pattern):
    pattern = re.compile(pattern + r'_(\d+)_(\d+)_(\d+)\.npy')
    heatmaps_dict = {}
    for filename in os.listdir(folder):
        match = pattern.match(filename)
        if match:
            i, j, k = map(int, match.groups())
            filepath = os.path.join(folder, filename)
            data = np.load(filepath)
            
            if i not in heatmaps_dict:
                heatmaps_dict[i] = []
            
            while len(heatmaps_dict[i]) <= j:
                heatmaps_dict[i].append([])
            
            while len(heatmaps_dict[i][j]) <= k:
                heatmaps_dict[i][j].append(None)
            
            heatmaps_dict[i][j][k] = data
    return [heatmaps_dict[i] for i in sorted(heatmaps_dict.keys())]

In [ ]:
import os

# Path to your folder
folder = "savefiles/"

for filename in os.listdir(folder):
    if "_Actor_" in filename:
        new_name = filename.replace("_Actor_", "_")
        old_path = os.path.join(folder, filename)
        new_path = os.path.join(folder, new_name)
        os.rename(old_path, new_path)
        print(f"Renamed: {filename} → {new_name}")